In [9]:
import pandas as pd
import time

In [10]:
it_cv_sample = pd.read_csv('data/processed/base_results/cvs_it_test_scores_advanced_LLM.csv').sample(n=10, random_state=42)

In [ ]:
reviewer_scores = []  # To store scores as integers

for i, row in it_cv_sample.iterrows():
    
    print("\n========================")
    print(f"Resume #{i + 1}:\n")
    print(row["Resume_str"])
    print("\n========================")
    time.sleep(0.1)
    time.sleep(0.1)

    while True:
        try:
            score = int(input("Your score (0–10): "))
            if 0 <= score <= 10:
                reviewer_scores.append(score)
                break
            else:
                print("⚠️ Please enter a number between 0 and 10.")
        except ValueError:
            print("⚠️ Invalid input. Please enter an integer between 0 and 10.")

print("\n✅ Review complete. Here are your scores:")
print(reviewer_scores)


Resume #84:

         IT MANAGEMENT           Career Overview    Detail-oriented professional with extensive Information Technology experience in hardware and software troubleshooting/management. With additional experience in networking, task automation, server technologies, digital graphic design, client + server side programming technologies and popular industry software packages including Microsoft Office and Adobe Creative Suite.      Qualifications        Education and certificates          Troubleshooting q   CompTIA A+ certified (2012)          q   Advanced troubleshooting q   CompTIA Net+ certification class       q   Virus and spyware removal q   MCSA certification class          q   Computer diagnostics and repair q   Web Development I + II class          q   Gaming console repair and q   Digital media manipulation class          modification q   Java programming class          q   Mobile device repair q   Hardware support and Programming and Web          troubleshooting q  